In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import os
import gc
from tqdm import tqdm
import random

import warnings
warnings.filterwarnings('ignore')

In [2]:
import keras
import tensorflow as tf
from tensorflow.keras import optimizers, callbacks
from tensorflow.keras.layers import Dense, Concatenate, Activation, Add, BatchNormalization, Dropout, Input
from tensorflow.keras.models import Model, Sequential, load_model

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)

def mish(x):
    return x*tf.math.tanh(tf.math.softplus(x))

def decay(epochs):
    init = 1e-3
    drop = 10
    ratio = 0.9
    return max(5e-5, (init * (ratio ** (epochs//drop))))

es = callbacks.EarlyStopping(patience=10, restore_best_weights=True)
lrs = callbacks.LearningRateScheduler(decay, verbose=0)


Using TensorFlow backend.


In [3]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

data = load_breast_cancer()
data, target = data['data'], data['target']

tr_X, val_X, tr_y, val_y = train_test_split(data, target, test_size=0.2, random_state=SEED)

In [4]:
scaler = StandardScaler()
scaler.fit(tr_X)

tr_X = scaler.transform(tr_X)
val_X = scaler.transform(val_X)

In [5]:
def DenseBlock(n, activation=None, bn=True):
    def f(x):
        x = Dense(n, kernel_initializer='he_normal')(x)
        if bn:
            x = BatchNormalization()(x)
        x = Activation(activation)(x)
        return x
    return f


def build_StackNN(input_shape):
    inputs = Input(shape=input_shape)
    
    x1 = DenseBlock(32, activation=mish)(inputs)
    x1 = DenseBlock(16, activation=mish)(x1)
    x1_out = Dense(1, activation='sigmoid')(x1)
    
    x2 = DenseBlock(32, activation=mish)(inputs)
    x2 = DenseBlock(16, activation=mish)(x2)
    x2_out = Dense(1, activation='sigmoid')(x2)
    
    x3 = DenseBlock(32, activation=mish)(inputs)
    x3 = DenseBlock(16, activation=mish)(x3)
    x3_out = Dense(1, activation='sigmoid')(x3)
    
    x = Concatenate()([x1, x2, x3])
    x = DenseBlock(16)(x)
    x_out = Dense(1, activation='sigmoid', name='tot')(x)
    
    return Model(inputs, [x_out, x1_out, x2_out, x3_out])

In [6]:
nn = build_StackNN(tr_X.shape[1:])
nn.compile(loss='binary_crossentropy', optimizer=optimizers.Adam())

es = callbacks.EarlyStopping(patience=10, restore_best_weights=True, monitor='val_tot_loss')

In [7]:
nn.fit(tr_X, [tr_y]*4,
      epochs=150,
      validation_split=0.15,
      callbacks=[es, lrs])

Train on 386 samples, validate on 69 samples
Epoch 1/150
386/386 [==============================] - 2s 6ms/sample - loss: 2.3215 - tot_loss: 0.5745 - dense_2_loss: 0.5505 - dense_5_loss: 0.6116 - dense_8_loss: 0.6657 - val_loss: 1.9599 - val_tot_loss: 0.4718 - val_dense_2_loss: 0.3711 - val_dense_5_loss: 0.5039 - val_dense_8_loss: 0.7875
Epoch 2/150
386/386 [==============================] - 0s 596us/sample - loss: 1.7084 - tot_loss: 0.4317 - dense_2_loss: 0.4795 - dense_5_loss: 0.4524 - dense_8_loss: 0.5198 - val_loss: 1.4003 - val_tot_loss: 0.2782 - val_dense_2_loss: 0.3203 - val_dense_5_loss: 0.3086 - val_dense_8_loss: 0.5502
Epoch 3/150
386/386 [==============================] - 0s 627us/sample - loss: 1.3841 - tot_loss: 0.3343 - dense_2_loss: 0.4255 - dense_5_loss: 0.3923 - dense_8_loss: 0.4518 - val_loss: 1.1288 - val_tot_loss: 0.2191 - val_dense_2_loss: 0.2830 - val_dense_5_loss: 0.2354 - val_dense_8_loss: 0.4042
Epoch 4/150
386/386 [==============================] - 0s 634us/sa

386/386 [==============================] - 0s 575us/sample - loss: 0.4539 - tot_loss: 0.0724 - dense_2_loss: 0.1227 - dense_5_loss: 0.1040 - dense_8_loss: 0.1277 - val_loss: 0.5294 - val_tot_loss: 0.0974 - val_dense_2_loss: 0.1452 - val_dense_5_loss: 0.1187 - val_dense_8_loss: 0.1412
Epoch 29/150
386/386 [==============================] - 0s 570us/sample - loss: 0.4316 - tot_loss: 0.0665 - dense_2_loss: 0.1195 - dense_5_loss: 0.0989 - dense_8_loss: 0.1205 - val_loss: 0.5239 - val_tot_loss: 0.0959 - val_dense_2_loss: 0.1431 - val_dense_5_loss: 0.1171 - val_dense_8_loss: 0.1391
Epoch 30/150
386/386 [==============================] - 0s 557us/sample - loss: 0.4440 - tot_loss: 0.0722 - dense_2_loss: 0.1199 - dense_5_loss: 0.1167 - dense_8_loss: 0.1227 - val_loss: 0.5202 - val_tot_loss: 0.0952 - val_dense_2_loss: 0.1411 - val_dense_5_loss: 0.1164 - val_dense_8_loss: 0.1368
Epoch 31/150
386/386 [==============================] - 0s 619us/sample - loss: 0.5543 - tot_loss: 0.1704 - dense_2_los

In [8]:
roc_auc_score(val_y, nn.predict(val_X)[0])

0.9944317065181788